In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn import neighbors, datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# import some data to play with
iris = datasets.load_iris()

# Index

* [About classification algorithms](#About-classification-algorithms)
* [Train and test subsamples](#Train-and-test-subsamples)
* [K nearest neigbors](#KNN)
* [Decision Trees](#Decision-Trees)
    * [Splitting the dataset](#Splitting-the-dataset)
    * [Avoiding overfitting](#Avoiding-overfitting)

## About classification algorithms
Some classification algorithms can only distinguish between two classes, how can we use them in multi class problems? There are two approaches to this:
    
* **One vs one:** is the approach where we evaluate the classes in pairs. Say we have three classes, A, B and C. The OVO ensemble will be composed of 3 (= 3 * (3 - 1) / 2) binary classifiers. The first will discriminante A from B, the second A from C, and the third B from C. At prediction stage, the class that got the highest number of "+1" predictions is our winner. Notice that this is a $O(n^2)$ problem
      
* **One vs rest:** (aka one-vs-all)is the strategy that involves training one classifier (estimator) for class and then taking the one which gives the highest confidence.

[wiki](https://en.wikipedia.org/wiki/Multiclass_classification#Transformation_to_binary)

## Train and test subsamples
In general we should split the data given in two parts: one for training and the other for testing. Usually the testing slice is 1/3 of the dataset.

## KNN
K nearest neigbors is a *lazy* algorithm which does not learn and makes computations in classification time, that is, find a predefined number of training samples (k) closest in distance to the new point, and predict the label from these.

Notice that KNN takes by default the k closest samples regardless how far they are, to mitigate this effect a weight parameter can be added.

KNN can also be applied to time series but they're pretty much regression problems we'll see them in due time. 

**Key features of KNN:**
* Easy to understand and implement.
* Computationally efficient in general (with small datasets).
* Defining similarities.
* The first thing that should be tried when approaching a ML problem.
* They suffer especially the [curse of dimensionality](./../Glossary.ipynb/#C).

In [ ]:
##### PART #1, Load an preprocess the data #####

# we only take the first two features in the dataset
X = iris.data[:, :2]
cols = iris['feature_names'][:2]
y = iris.target


##### PART 2, create the model #####

# Number of neighbors and weight
k, w = 30, 'distance'

# we create an instance of Neighbours Classifier and fit the data.
clf = neighbors.KNeighborsClassifier(n_neighbors=k, weights=w)
clf.fit(X, y)


##### PART #3, plot the outcome ####

# We are about to create a mesh of points that will represent a bunch of predictions 
h = .01  # step size in the mesh
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

# Once created the mesh, drop all the points into the model and predict the values for them
Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

# Plot the prediction areas (background)
cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
Z = Z.reshape(xx.shape)  # reshape to match the grid, same as yy.shape
plt.figure()
plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

# Plot also the training points (real points)
cmap_bold =  ListedColormap(['#FF0000', '#00FF00', '#0000FF'])
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=cmap_bold, edgecolor='k', s=20)
plt.xlabel(cols[0])
plt.ylabel(cols[1])
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.title("3-Class classification (k = %i)"% (k))

plt.show()

## Decision Trees
[Nice visualization](http://www.r2d3.us/visual-intro-to-machine-learning-part-1/) 

Decision trees divide the space into high dimensional rectangles. They are simple to understand and interpret (white box model), but they tend to overfit the data. However, they are useful in other ML techniques like bagging or random forests.

In [ ]:
# We take the features in pairs (Uncomment to see other pairs)
pair = [0,1]
#pair = [1,2] 
#pair = [2,3] 
X = iris.data[:, pair]
y = iris.target

# Train
clf = DecisionTreeClassifier().fit(X, y)

# Display the score (in the same training set, notice)
print('score was: {}'.format(clf.score(X, y)))

# Again, create a mesh of points that will represent a bunch of predictions
plot_step = 0.02
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),np.arange(y_min, y_max, plot_step))
plt.tight_layout(h_pad=0.5, w_pad=0.5, pad=2.5)

# Once created the mesh, drop all the points into the model and predict the values for them
Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

plt.xlabel(iris.feature_names[pair[0]])
plt.ylabel(iris.feature_names[pair[1]])

# Plot the training points
plot_colors, n_classes = 'ryb', 3
for i, color in zip(range(n_classes), plot_colors):
    idx = np.where(y == i)
    plt.scatter(X[idx, 0], X[idx, 1], c=color, label=iris.target_names[i], edgecolor='black', s=15)
plt.show()

### Splitting the dataset
Let's split the dataset into training and testing subsamples so we can check how effective is our training

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, stratify=y)

# Train
clf = DecisionTreeClassifier().fit(X_train, y_train)

# Display the score
print('score was: {}'.format(clf.score(X_test, y_test)))

# Plot the decision boundary
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),np.arange(y_min, y_max, plot_step))
plt.tight_layout(h_pad=0.5, w_pad=0.5, pad=2.5)

Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

plt.xlabel(iris.feature_names[pair[0]])
plt.ylabel(iris.feature_names[pair[1]])

# Plot the training points
for i, color in zip(range(n_classes), plot_colors):
    idx = np.where(y_test == i)
    plt.scatter(X_test[idx, 0], X_test[idx, 1], c=color, label=iris.target_names[i], edgecolor='black', s=15)
plt.show()


### Avoiding overfitting
As we can see, decision trees tend to overfit to the trining data, there are two ways to mitigate this:
* Bagging **(B**ootstrap **agg**regat**ing**) [[wiki]](https://en.wikipedia.org/wiki/Bootstrap_aggregating)
* Random Forests [[wiki]](https://en.wikipedia.org/wiki/Random_forest)

**A: Bagging:** take several random subsets of the data, train them independently and finally aggregate the resutls and vote the best one.

**B: Random Forests:** Instead of taking random subsets of the data, we take random subsets of the features.

In [ ]:
# Create an artificial dataset
X, y = datasets.make_classification(n_samples=10000, n_features=6,
                           n_informative=2, n_redundant=0,
                           random_state=0, shuffle=False)

# Now split the train and the test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, stratify=y)

# Train the model
clf = RandomForestClassifier(
    n_estimators=100, max_depth=2, random_state=0)
clf.fit(X_train, y_train)  

# Display the score
print('score was: {}'.format(clf.score(X_test, y_test)))


print(clf.feature_importances_)

print(clf.predict([[0, 0, 0, 0, 0, 0]]))